# Reference: https://leimao.github.io/article/Neural-Networks-Quantization/#Floating-Point-Quantization

In [1]:
import numpy as np

### 1.Hàm tính giá trị độ co giãn 
---

$$s = \frac{\beta - \alpha}{\beta_q - \alpha_q} $$


In [2]:
def calculate_s(alpha, beta, alpha_q, beta_q):
    """
    Hàm thực hiện tính giá trị độ co giãn
    Đầu vào:
        alpha: Giá trị nhỏ nhất của miền hiện tại
        beta: Giá trị lớn nhất của miền hiện tại
        alpha_q: Giá trị nhỏ nhất của miền lượng tử hóa
        beta_q: Giá trị lớn nhất của miền lượng tử hóa
    Đầu ra:
        s: Giá trị độ co giãn
    """
    # TODO 1: Tính giá trị độ co giãn
    s = (beta - alpha) / (beta_q - alpha_q)
    return s

Test code

In [3]:
try:
    alpha = -5.0
    beta = 10.0
    alpha_q = -30
    beta_q = 50
    print('s: {}'.format(calculate_s(alpha, beta, alpha_q, beta_q)))
except Exception as e:
    print("Lỗi thực thi: {}", e)

s: 0.1875


**Kết quả mong đợi**

s: 0.1875

### 2. Tính giá trị điểm 0
---

$z = \text{round}(z) = \text{round}(\frac{\beta\alpha_q - \alpha\beta_q}{\beta - \alpha})$. 

In [4]:
def calculate_z(alpha, beta, alpha_q, beta_q):
    """
    Hàm thực hiện tính giá trị điểm 0
    Đầu vào:
        alpha: Giá trị nhỏ nhất của miền hiện tại
        beta: Giá trị lớn nhất của miền hiện tại
        alpha_q: Giá trị nhỏ nhất của miền lượng tử hóa
        beta_q: Giá trị lớn nhất của miền lượng tử hóa
    Đầu ra:
        z: Giá trị điểm 0
    """
    # TODO 2: Tính giá trị điểm 0
    z = round((beta*alpha_q - alpha*beta_q) / (beta - alpha))
    return z

Test code

In [5]:
try:
    alpha = -5.0
    beta = 10.0
    alpha_q = -30
    beta_q = 50
    print('z: {}'.format(calculate_z(alpha, beta, alpha_q, beta_q)))
except Exception as e:
    print("Lỗi thực thi: {}", e)

z: -3


**Kết quả mong đợi**

z: -3

### 3. Hàm tính đồng thời s và z
---

Ví dụ các giá trị kiểu số nguyên 8 (int8) sẽ là số nguyên nằm trong đoạn $[-2^7, 2^7-1] = [-128, 127]$

In [6]:
def calculate_s_and_z_int(alpha, beta, exp):
    """
    Hàm thực hiện tính giá trị độ co giãn và điểm 0
    Đầu vào:
        alpha: Giá trị nhỏ nhất của miền hiện tại
        beta: Giá trị lớn nhất của miền hiện tại
        exp: Giá trị bit đại diện cho miền số nguyên mục tiêu
            - Ví dụ exp = 8 có nghĩa là miền mục tiêu là int8
    Đầu ra:
        s, z: giá trị độ co giãn và điểm 0
    """

    # TODO 3: Tìm ra miền số nguyên mục tiêu
    alpha_q = -(2**(exp-1))
    beta_q = 2**(exp-1) -1
    # TODO 4: Tính s và z sử dụng những hàm bên trên
    s = calculate_s(alpha, beta, alpha_q, beta_q)
    z = calculate_z(alpha, beta, alpha_q, beta_q)
    return s, z

Test code

In [7]:
try:
    alpha = -2.0
    beta = 3.0
    s_test, z_test = calculate_s_and_z_int(alpha, beta, 8)
    print('s_test: {}'.format(s_test))
    print('z_test: {}'.format(z_test))
except Exception as e:
    print("Lỗi thực thi: {}", e)

s_test: 0.0196078431372549
z_test: -26


**Kết quả mong đợi** (Xấp xỉ)

s_test: 0.0196078431372549 \
z_test: -26


### 4. Hàm lượng tử hóa một giá trị về miền số nguyên
---

$$x_q = \text{clip}\Big( \text{round}\big(\frac{1}{s} x + z\big), \alpha_q, \beta_q \Big)$$

In [8]:
def quantize_int(x, s, z, exp):
    """
    Hàm thực hiện lượng tử hóa
    Đầu vào:
        x: giá trị muốn lượng tử hóa
    Đầu ra:
        x_q: giá trị đã lượng tử hóa
    """
    # TODO 5: Tìm ra miền số nguyên mục tiêu
    alpha_q = -(2**(exp-1))
    beta_q = 2**(exp-1) -1
    # Áp dụng công thức lượng tử 
    # Sau đó làm tròn giá trị
    # Ví dụ 3.2 còn 3.0 và 3.6 thành 4.0
    x_q = np.round((1/s)*x + z)
    # Cắt giá trị nếu nằm ngoài miền mục tiêu
    x_q = np.clip(x_q, alpha_q, beta_q)
    # Ép về miền tương ứng
    if exp == 8:
        x_q = x_q.astype(np.int8)
    else:
        x_q = x_q.astype(np.int16)
    return x_q

Test Code

In [9]:
try:
    exp = 8
    alpha = -100
    beta = 200
    s, z = calculate_s_and_z_int(alpha, beta, exp)
    x = 10
    x_q = quantize_int(x, s, z, exp)
    print('x_q: {}'.format(x_q))
except Exception as e:
    print("Lỗi thực thi: {}", e)

x_q: -34


**Kết quả mong đợi** 

x_q: -34


### 5. Hàm khởi tạo một ma trận ngẫu nhiên các giá trị theo phân phối đều (Uniform) kiểu float32
---

In [10]:
def create_uniform_matrix(size, low, high):
    """
    Hàm tạo một ma trận bất kỳ
    Đầu vào:
        size: Chiều của ma trận
            - Ví dụ (2, 3)
        low: Giá trị nhỏ nhất
        high: Giá trị lớn nhất
    Đầu ra:
        matrix: Ma trận với những giá trị ngẫu nhiên được khởi tạo
    """
    matrix = np.random.uniform(low=low, high=high, size=size).astype(np.float32)
    return matrix

In [11]:
try:
    random_seed = 1
    np.random.seed(random_seed)
    matrix = create_uniform_matrix((2, 3), 5.0, 10.0)
    print(matrix)
except Exception as e:
    print("Lỗi thực thi: {}", e)

[[7.08511   8.601623  5.0005717]
 [6.511663  5.7337794 5.461693 ]]


**Kết quả mong đợi** (Xấp xỉ)

```python
[[7.08511   8.601623  5.0005717]
 [6.511663  5.7337794 5.461693 ]]
```


### 6. Hàm lượng tử hóa một giá trị trong ma trận
---

Phép nhân ma trận có dạng

$$
Y = XW + b
$$

Với $$X \in \mathbb{R}^{m \times p}$$ $$W \in \mathbb{R}^{p \times n}$$ và $$b \in \mathbb{R}^{n}$$


Thiết kế hàm lượng tử một giá trị trong ma trận $Y$

- $i$: vị trí dòng
- $j$: vị trí cột


$$
\begin{align} Y_{q,i,j} &= z_Y + \frac{s_b}{s_Y} (b_{q, j} - z_b) \\ &\qquad + \frac{s_X s_W}{s_Y} \Bigg[ \bigg( \sum_{k=1}^{p} X_{q,i,k} W_{q, k,j} \bigg) - \bigg( z_W \sum_{k=1}^{p} X_{q,i,k} \bigg) - \bigg( z_X \sum_{k=1}^{p} W_{q, k,j} \bigg) + p z_X z_W\Bigg] \end{align}
$$

In [12]:
def quantize_a_position_in_matrix_int(X_q, W_q, b_q, z_X, z_W, z_b, s_X, s_W, s_b, s_Y, i, j, p, exp):
    """
    Hàm lượng tử hóa một giá trị của kết quả nhân ma trận
    Đầu vào:
        X_q: ma trận X đã lượng tử
            - Chiều: (m, p)
        W_q: ma trận W đã lượng tử
            - Chiều: (p, n)
        b_q: vector b đã lượng tử
            - Chiều: (1, n)
        z_X: giá trị điểm 0 miền mục tiêu X
        z_W: giá trị điểm 0 miền mục tiêu W
        z_b: giá trị điểm 0 miền mục tiêu b
        s_X: giá trị độ co giãn miền mục tiêu X
        s_W: giá trị độ co giãn miền mục tiêu W
        s_b: giá trị độ co giãn miền mục tiêu b
        s_Y: giá trị độ co giãn miền mục tiêu Y
        i: Hàng của vị trí
        j: Cột của vị trí
        p: Chiều p
        exp: Số mũ của miền mục tiêu
            - Ví dụ 8 có ý nghĩa miền mục tiêu là int8
    Đầu ra:
        quantized_Y_i_j: Giá trị đã được lượng tử về miền số nguyên mục tiêu
    """

    # Tính các giới hạn của miền
    low = -(2**(exp-1))
    high = 2**(exp-1) - 1

    # TODO 6: Áp dụng công thức lượng tử hóa. Chú ý ép các giá trị của W_q, X_q và b_q về int32 để mở rộng miền tính toán
    W_q = W_q.astype(np.int32)
    X_q = X_q.astype(np.int32)
    b_q = b_q.astype(np.int32)
    z_Y = 0
    quantized_Y_i_j = z_Y + \
                        ((s_b / s_Y) * (b_q[0][j] - z_b)) + \
                        (((s_X * s_W)/(s_Y)) *(np.sum(X_q[i]*W_q[:,j]) - (z_W*np.sum(X_q[i])) - (z_X*np.sum(W_q[:,j])) + p*z_X*z_W))
    # Làm tròn giá trị
    quantized_Y_i_j = np.round(quantized_Y_i_j)
    # TODO 7: Cắt giá trị trong miền int tương ứng
    quantized_Y_i_j = np.clip(quantized_Y_i_j, low, high)
    # Nén về dạng int tương tứng
    if exp == 8:
        quantized_Y_i_j = quantized_Y_i_j.astype(np.int8)
    else:
        quantized_Y_i_j = quantized_Y_i_j.astype(np.int16)
    return quantized_Y_i_j

Test code

In [13]:
try:
    random_seed = 1
    np.random.seed(random_seed)

    # Chiều
    m = 3
    p = 4
    n = 5

    # ===== X =====
    alpha_X = -100.0
    beta_X = 100.0
    s_X, z_X = calculate_s_and_z_int(alpha=alpha_X, beta=beta_X, exp=8)
    X = create_uniform_matrix(( m, p), low=alpha_X, high=beta_X)
    X_q = quantize_int(x=X, s=s_X, z=z_X, exp=8)

    # ===== W =====
    alpha_W = -40.0
    beta_W = 20.0
    s_W, z_W = calculate_s_and_z_int(alpha=alpha_W, beta=beta_W, exp=8)
    W = create_uniform_matrix((p, n), low=alpha_W, high=beta_W)
    W_q = quantize_int(x=W, s=s_W, z=z_W, exp=8)

    # ===== b =====
    alpha_b = -200.0
    beta_b = 120.0
    s_b, z_b = calculate_s_and_z_int(alpha=alpha_b, beta=beta_b, exp=8)
    b = create_uniform_matrix((1, n), low=alpha_b, high=beta_b)
    b_q = quantize_int(x=b, s=s_b, z=z_b, exp=8)

    # Y
    alpha_Y = -4000.0
    beta_Y = 4000.0
    s_Y, z_Y = calculate_s_and_z_int(alpha=alpha_Y, beta=beta_Y, exp=8)

    # Vị trí muốn tính
    i = 0
    j = 3
    exp = 8
    quantized_Y_i_j = quantize_a_position_in_matrix_int(X_q, W_q, b_q, z_X, z_W, z_b, s_X, s_W, s_b, s_Y, i, j, p, exp)
    print('quantized_Y_i_j: {}'.format(quantized_Y_i_j))


except Exception as e:
    print('e', e)

quantized_Y_i_j: 7


**Kết quả mong đợi** 

quantized_Y_i_j: 7



Viết hàm lượng tử hóa toàn bộ ma trận về số nguyên

In [14]:
def quantization_matrix_multiplication_int(X_q, W_q, b_q, z_X, z_W, z_b, s_X, s_W, s_b, s_Y, p, exp):
    """
    Hàm lượng tử hóa đầu ra của một phép nhân ma trận
    Đầu vào:
        X_q: ma trận X đã lượng tử
            - Chiều: (m, p)
        W_q: ma trận W đã lượng tử
            - Chiều: (p, n)
        b_q: vector b đã lượng tử
            - Chiều: (1, n)
        z_X: giá trị điểm 0 miền mục tiêu X
        z_W: giá trị điểm 0 miền mục tiêu W
        z_b: giá trị điểm 0 miền mục tiêu b
        s_X: giá trị độ co giãn miền mục tiêu X
        s_W: giá trị độ co giãn miền mục tiêu W
        s_b: giá trị độ co giãn miền mục tiêu b
        s_Y: giá trị độ co giãn miền mục tiêu Y
        p: Chiều p
        exp: Số mũ của miền mục tiêu
            - Ví dụ 8 có ý nghĩa miền mục tiêu là int8
    Đầu ra:
        quantized_Y: Ma trận đã được lượng tử về miền số nguyên mục tiêu
    """
    # Tính các giới hạn của miền
    low = -(2**(exp-1))
    high = 2**(exp-1) - 1

    p = W_q.shape[0]

    # TODO 8: Áp dụng công thức lượng tử trên toàn ma trận. Chú ý ép các giá trị của W_q, X_q và b_q về int32 để mở rộng miền tính toán
    W_q = W_q.astype(np.int32)
    X_q = X_q.astype(np.int32)
    b_q = b_q.astype(np.int32)

    quantized_Y = z_Y + ((s_b / s_Y) * (b_q - z_b)) + \
                ((s_X * s_W)/(s_Y)) * (X_q@W_q - z_W*np.sum(X_q, axis=1).reshape(-1,1) - z_X*np.sum(W_q, axis=0).reshape(1,-1) + p*z_X*z_W)

    # TODO 9: Làm tròn giá trị
    quantized_Y = np.round(quantized_Y)

    # TODO 10: Cắt giá trị trong miền int tương ứng
    quantized_Y = np.clip(quantized_Y, low, high)

    if exp == 8:
        quantized_Y = quantized_Y.astype(np.int8)
    else:
        quantized_Y = quantized_Y.astype(np.int16)

    return quantized_Y

Test code

In [15]:
try:
    random_seed = 1
    np.random.seed(random_seed)

    # Chiều
    m = 2
    p = 3
    n = 4

    # ===== X =====
    alpha_X = -100.0
    beta_X = 100.0
    s_X, z_X = calculate_s_and_z_int(alpha=alpha_X, beta=beta_X, exp=8)
    X = create_uniform_matrix(( m, p), low=alpha_X, high=beta_X)
    X_q = quantize_int(x=X, s=s_X, z=z_X, exp=8)

    # ===== W =====
    alpha_W = -40.0
    beta_W = 20.0
    s_W, z_W = calculate_s_and_z_int(alpha=alpha_W, beta=beta_W, exp=8)
    W = create_uniform_matrix((p, n), low=alpha_W, high=beta_W)
    W_q = quantize_int(x=W, s=s_W, z=z_W, exp=8)

    # ===== b =====
    alpha_b = -200.0
    beta_b = 120.0
    s_b, z_b = calculate_s_and_z_int(alpha=alpha_b, beta=beta_b, exp=8)
    b = create_uniform_matrix((1, n), low=alpha_b, high=beta_b)
    b_q = quantize_int(x=b, s=s_b, z=z_b, exp=8)

    # Y
    alpha_Y = -4000.0
    beta_Y = 4000.0
    s_Y, z_Y = calculate_s_and_z_int(alpha=alpha_Y, beta=beta_Y, exp=8)

    exp = 8
    quantized_Y_i_j = quantization_matrix_multiplication_int(X_q, W_q, b_q, z_X, z_W, z_b, s_X, s_W, s_b, s_Y, p, exp)
    print('quantized_Y_i_j: \n{}'.format(quantized_Y_i_j))
except Exception as e:
    print('e', e)

quantized_Y_i_j: 
[[111   7  19  46]
 [127  16 124   2]]


**Kết quả mong đợi** 

```python
[[111   7  19  46]
 [127  16 124   2]]

```

